In [1]:
import requests
import datetime

import numpy as np
import pandas as pd
from eval_bid_sample import _get, _post

In [2]:
_API_URL = "https://research-api.solarkim.com"

### !!!예측량 입찰 함수 정의!!!

In [3]:
def _post_bids(amounts:list = [0.0] * 24):
    """
    일단위 태양광 발전량 입찰. 시간별 24개의 발전량을 입찰하며 API가 호출된 시간에 따라 입찰 대상일이 결정된다. (https://research-api.solarkim.com/docs#tag/Competition-2023/operation/post_bids_cmpt_2023_bids_post 참고)
    """
    if amounts == None:
        print("ERROR: Check amount variable.\n")
    else:
        success = _post(f"{_API_URL}/cmpt-2023/bids", amounts)
        print(success)

### 입찰 결과 확인 함수 정의

In [4]:
def _get_bids_result(date:str):
    """
    더쉐어 예측 모델의 예측 결과 조회 (https://research-api.solarkim.com/docs#tag/Competition-2023/operation/get_bids_result_date_cmpt_2023_bid_results__date__get 참고)
    """
    if date == None or len(date) != 10:
        print('ERROR: Check date variable')
    else:
        bid_results = _get(f"{_API_URL}/cmpt-2023/bid-results/{date}")
    print(bid_results)
    return bid_results

### 기상 예측 결과 얻는 함수 정의

In [5]:
def _get_weathers_forecasts(date:str):
    """
    기상데이터 일단위 기상예측 데이터 조회 (https://research-api.solarkim.com/docs#tag/Competition-2023/operation/get_weathers_forecasts_date_bid_round_cmpt_2023_weathers_forecasts__date___bid_round__get 참고)
    """
    bid_round_10 = 1
    bid_round_17 = 2
    if date == None or len(date) != 10:
        print('ERROR: Check date variable')
    else:       
        weather_fcst_10 = _get(
            f"{_API_URL}/cmpt-2023/weathers-forecasts/{date}/{bid_round_10}"
        )

        weather_fcst_17 = _get(
            f"{_API_URL}/cmpt-2023/weathers-forecasts/{date}/{bid_round_17}"
        )

    print(weather_fcst_10)
    print(weather_fcst_17)
    return weather_fcst_10, weather_fcst_17

### 모델 예측 발전량 조회 함수 정의

In [6]:
def _get_gen_forecasts(date:str):
    """
    더쉐어 예측 모델의 예측 발전량 조회, 입찰대상일의 5가지 예측 모델의 예측 발전량 값을 취득한다 (https://research-api.solarkim.com/docs#tag/Competition-2023/operation/get_gen_forecasts_date_cmpt_2023_gen_forecasts__date___bid_round__get 참고)
    """
    bid_round_10 = 1
    bid_round_17 = 2

    if date == None or len(date) != 10:
        print('ERROR: Check date variable')
    else:
        gen_fcst_10 = _get(f"{_API_URL}/cmpt-2023/gen-forecasts/{date}/{bid_round_10}")

        gen_fcst_17 = _get(f"{_API_URL}/cmpt-2023/gen-forecasts/{date}/{bid_round_17}")

    print(gen_fcst_10)
    print(gen_fcst_17)
    return gen_fcst_10, gen_fcst_17

### 실제 기상 데이터 얻는 함수 정의

In [7]:
def _get_weathers_observeds(date:str):
    """
    기상데이터 일단위 기상관측데이터 조회, 당일에 대해 조회하면 현재시간 기준 24시간치 조회 (https://research-api.solarkim.com/docs#tag/Competition-2023/operation/get_weathers_observeds_date_cmpt_2023_weathers_observeds__date__get 참고)
    """
    if date == None or len(date) != 10:
        print('ERROR: Check date variable')
    else:    
        weather_obsv = _get(f"{_API_URL}/cmpt-2023/weathers-observeds/{date}")
    print(weather_obsv)
    return weather_obsv

### 국제표준시 -> 한국시간대로 바꾸는 함수 정의

In [8]:
def clacTimeStd2KR(time:str):
    dt = time
    dt_obj = datetime.datetime.strptime(dt, '%Y-%m-%dT%H:%M:%S%z')
    dt_obj = dt_obj - datetime.timedelta(hours = -9)

    format = '%Y-%m-%d %H:%M:%S'

    result =  datetime.datetime.strftime(dt_obj, format)
    result = result + '+09:00'

    return result

# Test

In [9]:
res=pd.read_csv("../result/gen17_incentive_weight1024.csv",encoding="cp949")

In [ ]:
-0.07612896,  1.69179457, 13.04265534, 21.76186929, 20.17942529,
       14.13697944, 27.9416048 , 20.44751025, 30.73367625, 12.01625192,
        4.51355347, -3.74665397, -4.44325625

In [13]:
amounts = [0,0,0,0,0,0,
          -0.07612896,  1.69179457, 13.04265534, 21.76186929, 20.17942529,
       14.13697944, 27.9416048 , 20.44751025, 30.73367625, 12.01625192,
        4.51355347, -3.74665397, -4.44325625,
           0,0,0,0,0]
print(len(amounts))

24


In [14]:
_post_bids(amounts)

True


In [64]:
date = '2023-10-23'
_get_bids_result(date)

[]


[]

In [12]:
date = "2023-11-17"
wf10, wf17 = _get_weathers_forecasts(date)
wf10_df = pd.DataFrame.from_records(wf10)
wf17_df = pd.DataFrame.from_records(wf17)
wf10_df['time'] = wf10_df['time'].apply(clacTimeStd2KR)
wf17_df['time'] = wf17_df['time'].apply(clacTimeStd2KR)
wf10_df.to_csv('../data/weather_forecasts/wf10_{0}.csv'.format(date), index = False)
wf17_df.to_csv('../data/weather_forecasts/wf17_{0}.csv'.format(date), index = False)

[{'time': '2023-11-16T16:00:00+00:00', 'cloud': 64.0, 'temp': 11.69, 'humidity': 61.0, 'ground_press': 1017.0, 'wind_speed': 11.2, 'wind_dir': 305.0, 'rain': 0.32, 'snow': 0.0, 'dew_point': 5.0, 'vis': 16.0934, 'uv_idx': 0.0, 'azimuth': 32.5174, 'elevation': -71.5717}, {'time': '2023-11-16T17:00:00+00:00', 'cloud': 71.0, 'temp': 11.8, 'humidity': 55.0, 'ground_press': 1017.0, 'wind_speed': 11.97, 'wind_dir': 303.0, 'rain': 0.0, 'snow': 0.0, 'dew_point': 4.44444, 'vis': 9.65606, 'uv_idx': 0.0, 'azimuth': 61.1458, 'elevation': -62.4471}, {'time': '2023-11-16T18:00:00+00:00', 'cloud': 76.0, 'temp': 10.62, 'humidity': 68.0, 'ground_press': 1017.0, 'wind_speed': 12.64, 'wind_dir': 314.0, 'rain': 0.31, 'snow': 0.0, 'dew_point': 3.88889, 'vis': 9.65606, 'uv_idx': 0.0, 'azimuth': 76.5384, 'elevation': -50.9475}, {'time': '2023-11-16T19:00:00+00:00', 'cloud': 79.0, 'temp': 10.11, 'humidity': 69.0, 'ground_press': 1017.0, 'wind_speed': 11.24, 'wind_dir': 315.0, 'rain': 0.62, 'snow': 0.0, 'dew_po

In [14]:
col_name = ['time', 'cloud', 'temp', 'humidity',
            'ground_press', 'wind_speed', 'wind_dir',
            'rain', 'snow', 'dew_point', 'vis',
            'uv_idx', 'azimuth', 'elevation']    

In [11]:
date = "2023-11-17"
gen_fcst_10, gen_fcst_17 = _get_gen_forecasts(date)
gen_fcst_10_df = pd.DataFrame.from_records(gen_fcst_10)
gen_fcst_17_df= pd.DataFrame.from_records(gen_fcst_17)
gen_fcst_10_df['time'] = gen_fcst_10_df['time'].apply(clacTimeStd2KR)
gen_fcst_17_df['time'] = gen_fcst_17_df['time'].apply(clacTimeStd2KR)
gen_fcst_10_df.to_csv('../data/gen_fcst_10_{0}.csv'.format(date), index = False)
gen_fcst_17_df.to_csv('../data/gen_fcst_17_{0}.csv'.format(date), index = False)

[{'time': '2023-11-16T16:00:00+00:00', 'model1': 0.0, 'model2': 0.0, 'model3': 0.0, 'model4': 0.0, 'model5': 0.0}, {'time': '2023-11-16T17:00:00+00:00', 'model1': 0.0, 'model2': 0.0, 'model3': 0.0, 'model4': 0.0, 'model5': 0.0}, {'time': '2023-11-16T18:00:00+00:00', 'model1': 0.0, 'model2': 0.0, 'model3': 0.0, 'model4': 0.0, 'model5': 0.0}, {'time': '2023-11-16T19:00:00+00:00', 'model1': 0.0, 'model2': 0.0, 'model3': 0.0, 'model4': 0.0, 'model5': 0.0}, {'time': '2023-11-16T20:00:00+00:00', 'model1': 0.0, 'model2': 0.0, 'model3': 0.0, 'model4': 0.0, 'model5': 0.0}, {'time': '2023-11-16T21:00:00+00:00', 'model1': 0.0, 'model2': 0.0, 'model3': 0.0, 'model4': 0.0, 'model5': 0.0}, {'time': '2023-11-16T22:00:00+00:00', 'model1': 0.0, 'model2': 0.0, 'model3': 0.000485629, 'model4': 0.0, 'model5': 0.0}, {'time': '2023-11-16T23:00:00+00:00', 'model1': 1.69392, 'model2': 4.90879, 'model3': 0.5168, 'model4': 0.322122, 'model5': 5.41806}, {'time': '2023-11-17T00:00:00+00:00', 'model1': 7.86439, 'm

In [11]:
date = "2023-10-25"
weather_obsv = _get_weathers_observeds(date)
weather_obsv_df = pd.DataFrame.from_records(weather_obsv)
weather_obsv_df.to_csv('../data/weather_obsvs/weather_obsv_{0}.csv'.format(date), index = False)

[]


In [11]:
date = "2023-10-25"
_get_bids_result(date)


[{'time': '2023-10-25T01:00:00+09:00', 'model1': {'error': 0.0, 'incentive': 0}, 'model2': {'error': 0.0, 'incentive': 0}, 'model3': {'error': 0.0, 'incentive': 0}, 'model4': {'error': 0.0, 'incentive': 0}, 'model5': {'error': 0.0, 'incentive': 0}}, {'time': '2023-10-25T02:00:00+09:00', 'model1': {'error': 0.0, 'incentive': 0}, 'model2': {'error': 0.0, 'incentive': 0}, 'model3': {'error': 0.0, 'incentive': 0}, 'model4': {'error': 0.0, 'incentive': 0}, 'model5': {'error': 0.0, 'incentive': 0}}, {'time': '2023-10-25T03:00:00+09:00', 'model1': {'error': 0.0, 'incentive': 0}, 'model2': {'error': 0.0, 'incentive': 0}, 'model3': {'error': 0.0, 'incentive': 0}, 'model4': {'error': 0.0, 'incentive': 0}, 'model5': {'error': 0.0, 'incentive': 0}}, {'time': '2023-10-25T04:00:00+09:00', 'model1': {'error': 0.0, 'incentive': 0}, 'model2': {'error': 0.0, 'incentive': 0}, 'model3': {'error': 0.0, 'incentive': 0}, 'model4': {'error': 0.0, 'incentive': 0}, 'model5': {'error': 0.0, 'incentive': 0}}, {'t

[{'time': '2023-10-25T01:00:00+09:00',
  'model1': {'error': 0.0, 'incentive': 0},
  'model2': {'error': 0.0, 'incentive': 0},
  'model3': {'error': 0.0, 'incentive': 0},
  'model4': {'error': 0.0, 'incentive': 0},
  'model5': {'error': 0.0, 'incentive': 0}},
 {'time': '2023-10-25T02:00:00+09:00',
  'model1': {'error': 0.0, 'incentive': 0},
  'model2': {'error': 0.0, 'incentive': 0},
  'model3': {'error': 0.0, 'incentive': 0},
  'model4': {'error': 0.0, 'incentive': 0},
  'model5': {'error': 0.0, 'incentive': 0}},
 {'time': '2023-10-25T03:00:00+09:00',
  'model1': {'error': 0.0, 'incentive': 0},
  'model2': {'error': 0.0, 'incentive': 0},
  'model3': {'error': 0.0, 'incentive': 0},
  'model4': {'error': 0.0, 'incentive': 0},
  'model5': {'error': 0.0, 'incentive': 0}},
 {'time': '2023-10-25T04:00:00+09:00',
  'model1': {'error': 0.0, 'incentive': 0},
  'model2': {'error': 0.0, 'incentive': 0},
  'model3': {'error': 0.0, 'incentive': 0},
  'model4': {'error': 0.0, 'incentive': 0},
  'mo